In [1]:
import os

In [2]:
%pwd

'/Users/harshadahake/Downloads/python2025/yt-course-e2e-project/End-to-End-ML-project---Youtube/research'

In [5]:
os.chdir('../')

In [6]:
%pwd

'/Users/harshadahake/Downloads/python2025/yt-course-e2e-project/End-to-End-ML-project---Youtube'

In [18]:
# define an entity (returned by function)
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [19]:
# Configuration manager
from src.projectDefault.constants import *
from src.projectDefault.utils.common import read_yaml, create_directories

# this needs compatibility with Python version (3.9 or earlier)
## this will help revise the steps till here, when I get to it!

# also, what to do in the future with new python versions?
## TBD


In [20]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config

    

In [21]:
import os
import urllib.request as request
import zipfile
from src.projectDefault.logging import logger
from src.projectDefault.utils.common import get_size

In [24]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} downloaded! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the xpi file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok = True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [27]:
## pipeline (for data ingestion!)

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-05-25 00:31:43,461: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-05-25 00:31:43,462: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-25 00:31:43,463: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-05-25 00:31:43,463: INFO: common: created directory at: artifacts]
[2025-05-25 00:31:43,464: INFO: common: created directory at: artifacts/data_ingestion]
[2025-05-25 00:31:43,834: INFO: 1258507886: artifacts/data_ingestion/data.zip downloaded! with following info: 
Connection: close
Content-Length: 25472
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "a4bfb500e92667f526438efe461411a299e54625e6d31beb2ed132c9e82ba309"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 713B:E7FB2:76558E:993417:6832C697
Accept-Ranges: bytes
Date: Sun, 2

In [ ]:
### Day 3 ^